# Align and remove under/overexposed images

This notebook removes any under/overexposed frames from timelapse experiments and aligns the images. 

### The aligned images will be used later in the object localisation and tracking steps. 

The structure of this notebook is:

1. Load images using the octopuslite dask loader.
2. Find over/underexposed images by measuring each channel and frame for average pixel intensity.
3. Select a reference channel to center the alignment on
4. Register alignment and save out transformation tensor
5. (Optional) Apply transformation matrix to all channels and save out images
6. Check images using Napari
7. Function to iterate over many experiments, many positions

In [1]:
import os
import glob
import enum
import numpy as np
from pystackreg import StackReg
from skimage.io import imsave
from tqdm.auto import tqdm
from octopuslite import DaskOctopusLiteLoader
from skimage import transform as tf
from tqdm.auto import tqdm

In [2]:
def image_generator(files, crop = None):
    """
    Generator function for iterative procesessing of image files
    """
    
    if crop is None:
        for filename in files:
            img = imread(filename)
            yield img
    else:
        for filename in files:
            img = imread(filename)
            img = crop_image(img, crop)
            yield img

## 1. Find images, organise and load using octopuslite

Define root directory and specific experiment and location to align

In [3]:
root_dir = '/home/nathan/data/kraken/ras'
expt = 'ND0022'
pos = 'Pos12'

Create new subdir for image files and move them all there. This is so that the miscelleanous non-image files (such as transformation matrices and tracking files) are easy to access later on and not lost amongst many single frame timelapse images.

In [4]:
image_path = f'{root_dir}/{expt}/{pos}/{pos}_images'
if not os.path.exists(image_path):
    os.mkdir(image_path)
    files = sorted(glob.glob(f'{root_dir}/{expt}/{pos}/*.tif'))
    for file in files:
        os.rename(file, file.replace(f'{pos}', f'{pos}/{pos}_images'))

Lazily load image array and associated information using dask octopuslite-loader and display channels found. Note the optional background removal is not invoked at this stage.

In [5]:
images = DaskOctopusLiteLoader(image_path, remove_background = False)
print([channel.name for channel in images.channels])

['BRIGHTFIELD', 'GFP', 'RFP', 'IRFP', 'MASK_B', 'MASK_A', 'MASK_IRFP', 'MASK_RFP', 'MASK_GFP', 'MASK_2CH']


## 2. Identify under/overexposed images and display average channel brightness

In [7]:
for channel in images.channels:
    if 'MASK' in channel.name:
        print(channel)

Channels.MASK_B
Channels.MASK_A
Channels.MASK_IRFP
Channels.MASK_RFP
Channels.MASK_GFP
Channels.MASK_2CH


In [8]:
%%time
# pixel range criteria
max_pixel, min_pixel = 200, 2
# set empty dict arrays for mean values 
mean_arrays = {}
# set for dodgy frames (only unique entries)
dodgy_frame_list = set([])
#iterate over channels
for channel in tqdm(images.channels, desc = f'Finding mean values of image channels'):
    if 'MASK' in channel.name:
        continue
    # find mean value of each frame in each channel
    mean_arrays[channel.name] = [np.mean(img) for img in tqdm(images[channel.name], position = 0)]
    # iterate over frames
    for frame, mean_value in tqdm(enumerate(mean_arrays[channel.name]), position = 0, total = len(mean_arrays[channel.name])):
        # check to see if mean frame pixel value meets criteria
        if max_pixel < mean_value or mean_value < min_pixel:
            # if so add to delete list
            dodgy_frame_list.add(frame)
# format delete list to only include single values
dodgy_frame_list = list(dodgy_frame_list)
print('Number of under/over-exposed frames:', len(dodgy_frame_list))

Finding mean values of image channels:   0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/1106 [00:00<?, ?it/s]

  0%|          | 0/1106 [00:00<?, ?it/s]

  0%|          | 0/1106 [00:00<?, ?it/s]

  0%|          | 0/1106 [00:00<?, ?it/s]

  0%|          | 0/1106 [00:00<?, ?it/s]

  0%|          | 0/1106 [00:00<?, ?it/s]

  0%|          | 0/1106 [00:00<?, ?it/s]

  0%|          | 0/1106 [00:00<?, ?it/s]

Number of under/over-exposed frames: 8
CPU times: user 4min 26s, sys: 10.4 s, total: 4min 36s
Wall time: 5min 55s


### 2a. Filter blanks from main image folder into separate directory

This step is optional as there is a parameter within `DaskOctopusLiteLoader` that filters the images, but employing that every time you load images is time consuming for large data sets

In [9]:
# check if blanks dir exists and make if not
if not os.path.exists(f'{root_dir}/{expt}/{pos}/{pos}_blanks'):
    os.mkdir(f'{root_dir}/{expt}/{pos}/{pos}_blanks')
# move blank images into this directory
for channel in images.channels:
    for f in images.files(channel.name):
        for i in dodgy_frame_list:
            if str(i).zfill(9) in f:
                os.rename(f, f.replace('_images', '_blanks'))
# reload image arrays now that blanks filtered
images = DaskOctopusLiteLoader(image_path, remove_background = False)
images['gfp']

dask.array<stack, shape=(1098, 1352, 1688), dtype=uint8, chunksize=(1, 1352, 1688), chunktype=numpy.ndarray>

## 3. Select reference image to base alignment around

Display the average intensities of each channel. The automatically-measured brightest channel isn't necessarily the best.

In [31]:
print('Average channel brightness for selection of reference image:')
for channel in images.channels:
    print(f'{channel.value}: {channel.name}:', np.mean(mean_arrays[channel.name]))

Average channel brightness for selection of reference image:
0: BRIGHTFIELD: 40.96403863128827
1: GFP: 45.97970142886046
2: RFP: 6.969645431239206
3: IRFP: 47.732877102157886


In [33]:
# manually select reference channel by adding index
reference_channel = images.channels[1]
# automatically select reference channel from max average pixel value (ie. brightest channel)
#reference_channel = images.channels[max([(channel.value, np.mean(mean_arrays[channel.name])) for channel in images.channels])[0]]
reference_channel.name

'GFP'

#### 3a. Set cropped area of reference image to base alignment around 
Cropping as alignment struggles on large arrays such as `shape = (1200,1353,1682)`, this step is optional but you will still need to run `.compute()` on the dask array to load the image into memory to perform the alignment.

In [37]:
%%time
# crop central window out of reference image
reference_image = DaskOctopusLiteLoader(image_path, 
                                        crop = (500, 500)
                                       )[reference_channel.name].compute()
reference_image.shape

Using cropping: (500, 500)
CPU times: user 4min 53s, sys: 3min 39s, total: 8min 32s
Wall time: 55.6 s


(1638, 500, 500)

## 4. Register alignment and save out transformation tensor
Transformation tensor is a 3D series of transformation matrices over time

In [42]:
%%time
# create operator using transformation type (translation)
sr = StackReg(StackReg.TRANSLATION) 

# register each frame to the previous as transformation matrices/tensor
transform_tensor = sr.register_stack(reference_image, reference = 'previous')

# save out transform tensor
np.save(f'{root_dir}/{expt}/{pos}/{reference_channel.name.lower()}_transform_tensor.npy', transform_tensor)

/home/nathan/analysis/miniconda3/envs/cellx/lib/python3.9/site-packages/pystackreg/pystackreg.py:379: UserWarning: Detected axis 2 as the possible time axis for the stack due to its low variability, but axis 0 was supplied for registration. Are you sure you supplied the correct axis?
  warnings.warn(


CPU times: user 3min 19s, sys: 3.78 s, total: 3min 23s
Wall time: 3min 23s


In [47]:
transform_tensor.shape

(1638, 3, 3)

## 5. (Optional) Apply transformation matrix to all channels and save out images in separate directory

Consumes a lot of time and space to replicate images with minor translational shifts, it is advised to just use the transform parameter in the `DaskOctopusLiteLoader`. 

In [14]:
%%time
### iterating over channels
# create aligned image dir if does not exist 
if not os.path.exists(f'{root_dir}/{expt}/{pos}/{pos}_aligned'):
    os.mkdir(f'{root_dir}/{expt}/{pos}/{pos}_aligned')
# iterate over channels
for channel in images.channels:
    #iterate over all images in channel
    for i in tqdm(range(len(transform_tensor)), 
                  desc = f'Aligning {channel.name.lower()} channel {channel.value+1}/{len(images.channels)}'):
        # load specific transform matrix for that frame
        transform_matrix = tf.EuclideanTransform(matrix = transform_tensor[i,...],
                                                 rotation = None)
        # transform image
        transformed_image = (tf.warp(images[channel.name][i,...].compute(), 
                                     transform_matrix, preserve_range=True)).astype(np.uint8)
        # set transformed image pathname by editing base dir
        fn = images.files(channel.name)[i].replace('_images', '_aligned')
        # save trans image out
        imsave(fn, transformed_image, check_contrast=False)

Aligning irfp channel 4/4: 100%|██████████| 1067/1067 [03:04<00:00,  5.79it/s]

CPU times: user 5min 24s, sys: 51.4 s, total: 6min 15s
Wall time: 11min 45s


## 6. Check alignment using Napari

In [53]:
import napari

In [56]:
aligned_images = DaskOctopusLiteLoader(image_path, 
                                       #crop = (1200,1600), 
                                       transforms = f'{root_dir}/{expt}/{pos}/gfp_transform_tensor.npy',
                                       remove_background=False)
viewer = napari.Viewer()
for channel in aligned_images.channels:
    viewer.add_image(aligned_images[channel.name], 
                     name = channel.name, 
                     blending = 'additive', 
                     contrast_limits = [0,255])

## Batch execute

Do all of the above but for many experiment IDs and many positions

In [17]:
import re
from skimage.io import imread

In [14]:
root_dir = '/home/nathan/data/kraken/ras'
expt_list = sorted([expt for expt in os.listdir(root_dir) 
                    if 'ND' in expt and os.path.isdir(os.path.join(root_dir, expt))], 
                    key = lambda x: [int(y) for y in re.findall(r'\d+', x)])
max_pixel = 200 
min_pixel = 2
alignment_channel = 'gfp'
crop_area = 500
save_out_images = False ### this does not save out a copy of the images, only the transformation matrix
overwrite = True ### this checks for any prexisting transformations and does not overwrite
frame_removal = True ### this boolean choice is for blank frame removal

In [15]:
expt_list

['ND0013',
 'ND0014',
 'ND0016',
 'ND0017',
 'ND0018',
 'ND0019',
 'ND0020',
 'ND0021',
 'ND0022']

In [18]:
### Iterate over all experiments defined in expt_list
for expt in tqdm(['ND0022']):#expt_list):
    # Find all positions in that experiment
    pos_list = [pos for pos in os.listdir(f'{root_dir}/{expt}') 
                if 'Pos' in pos 
                and os.path.isdir(f'{root_dir}/{expt}/{pos}')]        
    ### Iterate over all positions in that experiment
    for pos in tqdm(pos_list):

        ### check if overwrite param is false check if raw directory already created and if type of transform file already exists and decide whether to skip pos
        if not overwrite and os.path.exists (f'{root_dir}/{expt}/{pos}/{pos}_images') and glob.glob(f'{root_dir}/{expt}/{pos}/*transform*.npy'):
            print(glob.glob(f'{root_dir}/{expt}/{pos}/*transform*.npy'), f'file found, skipping {expt}/{pos}')
            continue

        print(f'Starting {expt}/{pos}')
        
        ### create new subdir of for raw files and move them all there
        image_path = f'{root_dir}/{expt}/{pos}/{pos}_images'
        if not os.path.exists(image_path):
            os.mkdir(image_path)
            files = sorted(glob.glob(f'{root_dir}/{expt}/{pos}/*.tif'))
            for file in files:
                os.rename(file, file.replace(f'{pos}', f'{pos}/{pos}_images'))
        
        if frame_removal:
            ### pre load files from raw file dir 
            images = DaskOctopusLiteLoader(image_path, remove_background= False)

            ### measure mean pixel value arrays and use to find under/over-exposed frames
            # set empty dict arrays for mean values 
            mean_arrays = {}
            # set for dodgy frames (only unique entries)
            dodgy_frame_list = set([])
            #iterate over channels
            for channel in tqdm(images.channels, desc = f'Finding mean values of image channels'):
                if 'MASK' in channel.name:
                    continue
                # find mean value of each frame in each channel
                mean_arrays[channel.name] = [np.mean(img) for img in image_generator(images.files(channel.name))]
                # iterate over frames
                for frame, mean_value in enumerate(mean_arrays[channel.name]):
                    # check to see if mean frame pixel value meets criteria
                    if max_pixel < mean_value or mean_value < min_pixel:
                        # if so add to delete list
                        dodgy_frame_list.add(frame)
            # format delete list to only include single values
            dodgy_frame_list = list(dodgy_frame_list)
            print('Number of under/over-exposed frames:', len(dodgy_frame_list))

            # check if blanks dir exists and make if not
            if not os.path.exists(f'{root_dir}/{expt}/{pos}/{pos}_blanks'):
                os.mkdir(f'{root_dir}/{expt}/{pos}/{pos}_blanks')
            # move blank images into this directory
            for channel in images.channels:
                for f in images.files(channel.name):
                    for i in dodgy_frame_list:
                        if str(i).zfill(9) in f:
                            os.rename(f, f.replace('_images', '_blanks'))

        # crop central window out of reference image with blanks removed
        reference_image = DaskOctopusLiteLoader(image_path, 
                                                crop = (crop_area, crop_area)
                                               )[alignment_channel].compute() 
        ### just gfp masks
        #reference_image = (reference_image == 1).astype(np.uint8)
        ### reverse order of image
        #reference_image = np.flip(reference_image, axis = 0)
        
        ### Register alignment
        print('Registering alignment for', pos, expt)
        # create operator using transformation type (translation)
        sr = StackReg(StackReg.TRANSLATION) 
        # register each frame to the previous as transformation matrices/tensor
        transform_tensor = sr.register_stack(reference_image, reference = 'previous', )
        
        ### switch orientation again back to norm
        #transform_tensor = np.flip(transform_tensor, axis = 0)
        
        ### clip transformation tensor to eliminate any rare jumps, (1688-1600)/2=44
        transform_tensor = np.clip(transform_tensor, a_max= 44, a_min = -44)
        
        # save out transform tensor
        np.save(f'{root_dir}/{expt}/{pos}/{alignment_channel}_clipped_transform_tensor.npy', transform_tensor)

        if save_out_images:
            ### Perform alignment
            # create aligned image dir if does not exist 
            if not os.path.exists(f'{root_dir}/{expt}/{pos}/{pos}_aligned'):
                os.mkdir(f'{root_dir}/{expt}/{pos}/{pos}_aligned')
            # iterate over channels
            for channel in images.channels:
                #iterate over all images in channel
                for i in tqdm(range(len(transform_tensor)), 
                              desc = f'Aligning {channel.name.lower()} channel {channel.value+1}/{len(images.channels)}'):
                    # load specific transform matrix for that frame
                    transform_matrix = tf.EuclideanTransform(matrix = transform_tensor[i,...],
                                                             rotation = None)
                    # transform image
                    transformed_image = (tf.warp(filtered_images[channel.name][i,...].compute(), 
                                                 transform_matrix, preserve_range=True)).astype(np.uint8)
                    # set transformed image pathname by editing base dir
                    fn = images.files(channel.name)[i].replace('_images', '_aligned')
                    # save trans image out
                    imsave(fn, transformed_image, check_contrast=False)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Starting ND0022/Pos13


Finding mean values of image channels:   0%|          | 0/6 [00:00<?, ?it/s]

Number of under/over-exposed frames: 7
Using cropping: (500, 500)
Registering alignment for Pos13 ND0022
Starting ND0022/Pos12


Finding mean values of image channels:   0%|          | 0/10 [00:00<?, ?it/s]

Number of under/over-exposed frames: 0
Using cropping: (500, 500)
Registering alignment for Pos12 ND0022


In [ ]:
fn = f'{root_dir}/{expt}/{pos}/mask_reversed_clipped_transform_tensor.npy'
tens = np.load(fn)
#         print(fn)
#         clip_tens = np.clip(tens, a_max= 10, a_min = None)

#         np.save(fn.replace('.npy', '_clipped.npy'), clip_tens)

In [104]:
fn = '/home/nathan/data/kraken/ras/ND0013/Pos4/mask_reversed_clipped_transform_tensor.npy'

In [110]:
tens = np.load(fn)


In [111]:
np.amin(tens)

-44.0

In [112]:
np.amax(tens)

16.150855961557795

In [113]:
for i in tens:
    print(i)

[[  1.           0.         -31.97928431]
 [  0.           1.         -44.        ]
 [  0.           0.           1.        ]]
[[  1.           0.         -23.74394114]
 [  0.           1.         -44.        ]
 [  0.           0.           1.        ]]
[[  1.           0.         -26.99326956]
 [  0.           1.         -44.        ]
 [  0.           0.           1.        ]]
[[  1.           0.         -28.70274259]
 [  0.           1.         -44.        ]
 [  0.           0.           1.        ]]
[[  1.           0.         -32.17283504]
 [  0.           1.         -44.        ]
 [  0.           0.           1.        ]]
[[  1.           0.         -30.58519166]
 [  0.           1.         -44.        ]
 [  0.           0.           1.        ]]
[[  1.           0.         -29.52344225]
 [  0.           1.         -44.        ]
 [  0.           0.           1.        ]]
[[  1.           0.         -34.33935011]
 [  0.           1.         -44.        ]
 [  0.           0.        

 [  0.           0.           1.        ]]
[[  1.           0.         -44.        ]
 [  0.           1.         -27.79718588]
 [  0.           0.           1.        ]]
[[  1.           0.         -44.        ]
 [  0.           1.         -26.78545514]
 [  0.           0.           1.        ]]
[[  1.           0.         -44.        ]
 [  0.           1.         -26.35724134]
 [  0.           0.           1.        ]]
[[  1.           0.         -44.        ]
 [  0.           1.         -26.53890972]
 [  0.           0.           1.        ]]
[[  1.           0.         -44.        ]
 [  0.           1.         -25.33525395]
 [  0.           0.           1.        ]]
[[  1.           0.         -44.        ]
 [  0.           1.         -25.15998545]
 [  0.           0.           1.        ]]
[[  1.           0.         -44.        ]
 [  0.           1.         -23.71702246]
 [  0.           0.           1.        ]]
[[  1.           0.         -44.        ]
 [  0.           1.       

 [  0.           0.           1.        ]]
[[  1.           0.         -13.10092944]
 [  0.           1.         -15.3134837 ]
 [  0.           0.           1.        ]]
[[  1.           0.         -15.92022649]
 [  0.           1.         -15.06601309]
 [  0.           0.           1.        ]]
[[  1.           0.         -16.3683829 ]
 [  0.           1.         -15.13144933]
 [  0.           0.           1.        ]]
[[  1.           0.         -13.5906732 ]
 [  0.           1.         -15.56231664]
 [  0.           0.           1.        ]]
[[  1.           0.         -21.30321147]
 [  0.           1.         -16.13001446]
 [  0.           0.           1.        ]]
[[  1.           0.         -18.73848661]
 [  0.           1.         -15.32901733]
 [  0.           0.           1.        ]]
[[  1.          0.        -16.6813748]
 [  0.          1.        -15.2698674]
 [  0.          0.          1.       ]]
[[  1.           0.         -19.36960743]
 [  0.           1.         -15.991

In [106]:
np.amin(tens)

-100.47055287431124

In [95]:
np.amax(tens)

16.150855961557795

In [91]:
for i in tens:
    print(i)

[[  1.           0.         -31.97928431]
 [  0.           1.         -97.15529676]
 [  0.           0.           1.        ]]
[[  1.           0.         -23.74394114]
 [  0.           1.         -96.50722295]
 [  0.           0.           1.        ]]
[[  1.           0.         -26.99326956]
 [  0.           1.         -97.53803001]
 [  0.           0.           1.        ]]
[[  1.           0.         -28.70274259]
 [  0.           1.         -97.81383087]
 [  0.           0.           1.        ]]
[[  1.           0.         -32.17283504]
 [  0.           1.         -98.66394718]
 [  0.           0.           1.        ]]
[[  1.           0.         -30.58519166]
 [  0.           1.         -99.65305332]
 [  0.           0.           1.        ]]
[[  1.           0.         -29.52344225]
 [  0.           1.         -99.54330981]
 [  0.           0.           1.        ]]
[[   1.            0.          -34.33935011]
 [   0.            1.         -100.26264565]
 [   0.            0.

 [  0.           0.           1.        ]]
[[  1.           0.         -46.36221807]
 [  0.           1.         -56.82860291]
 [  0.           0.           1.        ]]
[[  1.           0.         -45.30199221]
 [  0.           1.         -57.13136257]
 [  0.           0.           1.        ]]
[[  1.           0.         -48.59329277]
 [  0.           1.         -57.30635963]
 [  0.           0.           1.        ]]
[[  1.           0.         -50.03911216]
 [  0.           1.         -55.91880429]
 [  0.           0.           1.        ]]
[[  1.           0.         -50.84062589]
 [  0.           1.         -56.15710517]
 [  0.           0.           1.        ]]
[[  1.           0.         -49.97770817]
 [  0.           1.         -55.93307507]
 [  0.           0.           1.        ]]
[[  1.           0.         -51.75140349]
 [  0.           1.         -55.12365987]
 [  0.           0.           1.        ]]
[[  1.           0.         -53.97054831]
 [  0.           1.       

 [  0.           0.           1.        ]]
[[  1.           0.         -25.74821247]
 [  0.           1.          -8.74476628]
 [  0.           0.           1.        ]]
[[  1.           0.         -25.24799625]
 [  0.           1.          -9.64231017]
 [  0.           0.           1.        ]]
[[  1.           0.         -22.48839896]
 [  0.           1.          -8.75421781]
 [  0.           0.           1.        ]]
[[  1.           0.         -28.93127598]
 [  0.           1.         -10.12558005]
 [  0.           0.           1.        ]]
[[  1.           0.         -26.10663394]
 [  0.           1.          -9.43445901]
 [  0.           0.           1.        ]]
[[  1.           0.         -21.93877278]
 [  0.           1.          -9.2744728 ]
 [  0.           0.           1.        ]]
[[  1.           0.         -24.89330495]
 [  0.           1.         -10.48108424]
 [  0.           0.           1.        ]]
[[  1.           0.         -20.54636447]
 [  0.           1.       

 [ 0.          0.          1.        ]]
[[ 1.          0.          0.31552509]
 [ 0.          1.         -4.21623091]
 [ 0.          0.          1.        ]]
[[ 1.          0.          5.21216387]
 [ 0.          1.         -3.43003788]
 [ 0.          0.          1.        ]]
[[ 1.          0.         -2.17889757]
 [ 0.          1.         -4.17377755]
 [ 0.          0.          1.        ]]
[[ 1.          0.          6.13644951]
 [ 0.          1.         -3.2241011 ]
 [ 0.          0.          1.        ]]
[[ 1.          0.          4.39990332]
 [ 0.          1.         -2.70561055]
 [ 0.          0.          1.        ]]
[[ 1.          0.          1.5136897 ]
 [ 0.          1.         -3.02426561]
 [ 0.          0.          1.        ]]
[[ 1.          0.          3.28632024]
 [ 0.          1.         -2.44480366]
 [ 0.          0.          1.        ]]
[[ 1.          0.          0.12839079]
 [ 0.          1.         -3.09914938]
 [ 0.          0.          1.        ]]
[[ 1.          0

In [101]:
transform_tensor_fn = f'{root_dir}/{expt}/{pos}/mask_reversed_clipped_transform_tensor.npy'

transform_tensor_fn.replace('_clipped_', '_')

'/home/nathan/data/kraken/ras/ND0013/Pos4/mask_reversed_transform_tensor.npy'

In [102]:
expt_list = sorted([expt for expt in os.listdir(root_dir) 
                    if 'ND' in expt and os.path.isdir(os.path.join(root_dir, expt))], 
                    key = lambda x: [int(y) for y in re.findall(r'\d+', x)])
for expt in tqdm(expt_list):
    expt_dir = os.path.join(root_dir, expt)
    if os.path.exists(expt_dir):
        pos_list = sorted([pos for pos in os.listdir(expt_dir) 
                            if pos.startswith('Pos') 
                            and os.path.isdir(os.path.join(expt_dir, pos))], 
                            key = lambda x: [int(y) for y in re.findall(r'\d+', x)])
        for pos in tqdm(pos_list):
            transform_tensor_fn = f'{root_dir}/{expt}/{pos}/mask_reversed_clipped_transform_tensor.npy'
            ## laod tensor
            transform_tensor = np.load(transform_tensor_fn)
            ## save out as unclipped
            np.save(transform_tensor_fn.replace('_clipped_', '_'), transform_tensor)
            ## clip          
            clip_tensor = np.clip(transform_tensor, a_max= 44, a_min = -44)
            ##save clip
            np.save(transform_tensor_fn, clip_tensor)

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/14 [00:00<?, ?it/s]

  0%|          | 0/14 [00:00<?, ?it/s]

  0%|          | 0/14 [00:00<?, ?it/s]

  0%|          | 0/14 [00:00<?, ?it/s]

  0%|          | 0/14 [00:00<?, ?it/s]

  0%|          | 0/14 [00:00<?, ?it/s]

  0%|          | 0/14 [00:00<?, ?it/s]

FileNotFoundError: [Errno 2] No such file or directory: '/home/nathan/data/kraken/ras/ND0020/Pos0/mask_reversed_clipped_transform_tensor.npy'